In [3]:
from IPython.core.display import display, HTML 
from ipywidgets import interact, interact_manual, Dropdown, SelectMultiple, HBox, VBox, Button, Output, FloatText, IntText, IntRangeSlider, RadioButtons,IntProgress
from IPython.display import clear_output

from IPython.display import display
from IPython.core.display import display, HTML
from collections import Counter
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import requests

import numpy as np
import matplotlib.pyplot as plt
import six
import pandas as pd
import matplotlib
from matplotlib.pyplot import figure

#map
import folium
    

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [1]:
def fetch_data_roles():
    
    url='https://dh.gu.se/ws/expansion/list.php?tb=pers_role'
    
    resp = requests.get(url=url)
    
    list_roles=[]
    
    for name in [feat['name'] for feat in resp.json()['features']]:
        
        list_roles.append(name)
        
        
    return list_roles
        
        
        
    

In [1]:

#production, organization, person, place, pers_role
def fetch_data(keyword):

    
  
    url='https://dh.gu.se/ws/expansion/search.php?tb=' + keyword +'&max=10000'

    resp = requests.get(url=url)
    
    max_count=len(resp.json()['features'])
    
    f = IntProgress(min=0, max=max_count) # instantiate the bar
    display(f) # display the bar
    
    dictionaries=[]
    #for id in [feat['properties']['id'] for feat in resp.json()['features']]:
    for id in [feat['id'] for feat in resp.json()['features']]:
        info_url = 'https://dh.gu.se/ws/expansion/edit.php?tb=' + keyword + '&id=' + id + '&lang=sv'

        resp = requests.get(url=info_url)
    
        dictionary=resp.json()
    
        dictionaries.append(dictionary)

        f.value +=1
        
    f.value=max_count
    return dictionaries

    

In [11]:
#given specific ID

#id_type:
#productions_by_organization
#people_by_production
#events_by_production
#events_by_place
def get_data_by_id(id_type, id_number):
    if id_type=='productions_by_organization':
        id_type_string='production&ref=org_id&id='
    elif id_type=='events_by_production':
        id_type_string='event&ref=prod_id&id='
    elif id_type=='people_by_production':
        id_type_string='rel_pers_prod&ref=prod_id&id=' 
    elif id_type=='events_by_place':
        id_type_string='event&ref=place_id&id='
        
    elif id_type=='productions_by_person':
        id_type_string='rel_pers_prod&ref=pers_id&id='
        
    else:
        print('error in id_type string')

    url='https://dh.gu.se/ws/expansion/byreference.php?tb=' + id_type_string + str(id_number)
    #url_get_productions='https://dh.gu.se/ws/expansion/byreference.php?tb=production&ref=org_id&id=' + organization_id
    #url_get_people='https://dh.gu.se/ws/expansion/byreference.php?tb=rel_pers_prod&ref=prod_id&id=' + production_id

    #url="https://dh.gu.se/ws/expansion/search.php?tb=event&op=1&sstring="
    resp = requests.get(url=url)

    prod_org_specific = []
    #for id in [feat['properties']['id'] for feat in resp.json()['features']]:
    for id in [feat['id'] for feat in resp.json()['features']]:
        info_url = 'https://dh.gu.se/ws/expansion/edit.php?tb=' + id_type_string + id + "&lang=sv"
        resp = requests.get(url=info_url)
        prod_org_specific.append(resp.json())

    
    list_dictionaries=[]

    for prod in prod_org_specific:

        list_dictionaries.append(prod)
        #list_dictionaries.append(prod['features'][0])



    return list_dictionaries

In [12]:
#person
#organization
#
def get_data_by_id_single(id_type, id_number):
    
    #if id_type=='person':
    info_url= 'https://dh.gu.se/ws/expansion/edit.php?tb='+ id_type + '&id=' + str(id_number) + '&lang=sv'
    #elif id_type=='organization':
    #info_url= 'https://dh.gu.se/ws/expansion/edit.php?tb=' + id_type + '&id=' + str(id_number) + '&lang=sv'
        

    resp = requests.get(url=info_url)

    dictionary=resp.json()

    #dictionary=specific_object['features'][0]
    
    return dictionary

    """

    if dictionary['notes'][2] != '':
        display(HTML('Notes:'))
        display(HTML(dictionary['notes'][2]))

    if dictionary['article'][2] !='':
        display(HTML('Article:'))
        display(HTML(dictionary['article'][2]))
        
    """

In [ ]:
def get_organization_data_for_notebook():
    #get data --> only once:
    #get a list of all organizations:
    dictionaries_organizations=fetch_data('organization')
    
    display(HTML('<p>Organisationsdata hämtad. Nu genomgång av data och organiserar den för att göra den användbar.</p>'))
    
    #second progress bar
    max_count = len(dictionaries_organizations)

    f = IntProgress(min=0, max=max_count) # instantiate the bar
    
    display(f) # display the bar
    
    organization_list_of_tuples=[]

    organization_list_of_tuples_only_w_productions=[]

    organizations_w_productions_dictionaries=[]

    #låta databasen göra all filtrering. output resultatet i python. 
    for dictionary in dictionaries_organizations:
        
                
        #update: want the shorter name in tuple (saw especially when implementing selectMultiple that it is needed)

        name_org=dictionary['fields']['name']['value']
        split_name_org=name_org.split(',')

        name=split_name_org[0]

        org_id=dictionary['fields']['id']['value']

        organization_tuple=(name, org_id)

        organization_list_of_tuples.append(organization_tuple)

        productions_dictionaries=get_data_by_id('productions_by_organization', org_id)

        #now get a list of all organizations that has any productions
        if len(productions_dictionaries)>0:

            organization_list_of_tuples_only_w_productions.append(organization_tuple)

            #need these in a dictionary to later 
            organizations_w_productions_dictionaries.append(dictionary)
        
        f.value+=1
        
    display(HTML('<p>Färdigt!</p>'))
        
    return dictionaries_organizations, organization_list_of_tuples_only_w_productions, organizations_w_productions_dictionaries

In [1]:

def person_via_org_prod():
    #function describing what will happen when the user changes the organization:
    #populate the production dropdown with all productions that the organization has had
    def change_org(c):

        global organization_id
        organization_id=str(organization.value)


        list_dictionaries_productions=get_data_by_id('productions_by_organization', id_number=organization_id)

        #could add something here that makes it so when one choose a organization - only those productions that also have 
        #people

        list_w_tuples_org_specific_productions=[]
        for dictionary_production in list_dictionaries_productions:


            tuple_org_specific_productions=(dictionary_production['fields']['name']['value'], dictionary_production['fields']['id']['value'])
            list_w_tuples_org_specific_productions.append(tuple_org_specific_productions)

        #populate the option with the list of tuples (list of tuple so that one value (name) is 
        #shown in the dropdown while the value that is "saved" is the id which in turn is used to
        #fetch the people involved in the production - see later)
        production.options=list_w_tuples_org_specific_productions

        #if change the organization and production.options=[] --> set the person dropdown to [] also
        if production.options==():
            person.options=[]

    #function describing what will happen once change the production (once the productions dropdown 
    #has been populated depening on what organization the user choose):
    #get all the people that was part of the selected production into person dropdown.
    def change_prod(c):

        production_id=str(production.value)

        list_dictionaries_people=get_data_by_id('people_by_production', id_number=production_id)

        list_w_tuples_production_specific_persons=[]

        #access the information I want in the dropdown:
        for dictionary_people in list_dictionaries_people:

            #print(dictionary_people)
            #print(dictionary_people['fields']['pers_id']['value'])

            lastname_firstname_gender= dictionary_people['fields']['pers_id']['extra']
            #lastname_firstname_gender=dictionary_people['pers_id'][1]

            lastname_firstname_gender_split=lastname_firstname_gender.split(",")

            firstname_lastname=lastname_firstname_gender_split[1] + ' ' + lastname_firstname_gender_split[0]

            firstname_lastname_role= firstname_lastname + ', ' + str(dictionary_people['fields']['pers_role_id']['extra'])

            #tuple_production_specific_person=(firstname_lastname_role, dictionary_people['pers_id'][2])

            tuple_production_specific_person=(firstname_lastname_role, dictionary_people['fields']['pers_id']['value'])

            #tuple_production_specific_persons=(dictionary['pers_id'][1], dictionary['pers_id'][2])
            list_w_tuples_production_specific_persons.append(tuple_production_specific_person)


        person.options=list_w_tuples_production_specific_persons

    #output widgets - to be able to clear only that specific widget when click update_buton

    output_person_article = Output()
    output_person_in_production= Output()

    output_person_in_organization= Output()

    #output_person_production_specific= Output()

    #what happend when click the button:
    #here could organize different output widgets
    def update_func(button_c):


        #start with an article about the selected person (will disply if one is selected)
        with output_person_article:

            #if already a person selected and displayed --> clear the output
            clear_output()

            #maybe add answer to question also - only display information if box checked for instance.
            if person.value is None:
                print('no person selected')
            else:
                id_number=str(person.value)

                #function defined in "web_services_functions" Notebook
                dictionary=get_data_by_id_single('person', id_number)


                if dictionary['fields']['article']['value']!='':
                #if dictionary['article'][2] !='':
                    #display(HTML(dictionary['article'][2]))
                    display(HTML(dictionary['fields']['article']['value']))

        with output_person_in_production:

            clear_output()

            production_id=str(production.value)

            list_dictionaries_people=get_data_by_id('people_by_production', id_number=production_id)

            list_roles=[]

            for dictionary_people in list_dictionaries_people:


                #if dictionary_people['pers_id'][2]==person.value:
                if dictionary_people['fields']['pers_id']['value']==person.value:

                    #firstname_lastname_role= firstname_lastname + ', ' + dictionary_people['pers_role_id'][1]
                    #role=dictionary_people['pers_role_id'][1]
                    role=dictionary_people['fields']['pers_role_id']['extra']
                    list_roles.append(role)

            html_link='https://expansion.dh.gu.se/production/' + str(production_id)

            string_roles=', '.join(list_roles)
            display(HTML('<p style="font-size:14px;font-weight:bold;">Roll(er) inom <a href=' + html_link + ' target="blank">' + ' selekterad produktion</a>: </p>'))

            display(HTML('<p style="font-size:14px;margin-left:40px;">' +   string_roles +'</p>'))

        with output_person_in_organization:

            clear_output()

            person_id=person.value

            list_dictionaries_productions=get_data_by_id('productions_by_person',person_id)

            display(HTML('<p style="font-size:14px;font-weight:bold;">Medverkan i produktioner av selekterad organisation: </p>'))

            list_production_ids=[]
            list_production_names=[]
            list_production_roles=[]

            #added
            list_organizations=[]
            list_production_ids_not_org_specific=[]
            list_production_names_not_org_specific=[]
            list_production_roles_not_org_specific=[]

            #productions by specific person. looping over each. 
            for dictionary_production in list_dictionaries_productions:

                #if the the person have been in selected organization

                production_string=dictionary_production['fields']['prod_id']['extra']
                
                production_id=dictionary_production['fields']['prod_id']['value']
                
                #print(production_string)
                if production_string==None:
                    
                    html_link='https://expansion.dh.gu.se/production/' + str(production_id)
                    
                    display(HTML('<p style="font-size:14px;margin-left:40px;">Roll inom produktion som inte har namn speciferat: uppdatera databasen <a href=' + html_link +   ' target="_blank"> här!'))

                    
                    continue
                    

                #can't use split by ", " --> ex hamlet, 1992-00-00
                split_production_string=production_string.split(", ")

                #not always correct - get the final?
                #organization_name=split_production_string[2]
                organization_name=split_production_string[-1]

                #organization_id=organization.value



                selected_organization_dictionary=get_data_by_id_single('organization', id_number=organization_id)

                selected_organization=selected_organization_dictionary['fields']['name']['value']

                production_string=dictionary_production['fields']['prod_id']['extra']

                split_production_string=production_string.split(", ")

                production_name=split_production_string[0]

                if organization_name==selected_organization:
                    list_production_ids.append(production_id)

                    list_production_names.append(production_name)

                    list_production_roles.append(dictionary_production['fields']['pers_role_id']['extra'])

                    #added (for "med i övriga organisationer"):
                    list_organizations.append(organization_name)

                    #before only in else -- now want all (also selected production)
                    list_production_names_not_org_specific.append(production_name)
                    list_production_roles_not_org_specific.append(dictionary_production['fields']['pers_role_id']['value'])


                else:
                    list_organizations.append(organization_name)
                    list_production_ids_not_org_specific.append(production_id)
                    list_production_names_not_org_specific.append(production_name)
                    list_production_roles_not_org_specific.append(dictionary_production['fields']['pers_role_id']['value'])



            list_production_ids_sorted=[x for _,x in sorted(zip(list_production_ids,list_production_ids))]
            list_production_names_sorted=[x for _,x in sorted(zip(list_production_ids,list_production_names))]
            list_production_roles_sorted=[x for _,x in sorted(zip(list_production_ids,list_production_roles))]


            index=0
            count=0
            first=True
            #now happening once for each production - something is written.
            break_status=False
            #the problem is the last one - or if two same
            for production_id in list_production_ids_sorted:


                #happening for the first production in loop
                if first==True:
                    index_within=0
                    first=False

                #if arrive where previous "round" added to index_wihin - skip round and add to index
                #skip rounds until back to 0 for index_within.  
                if index_within>1:

                    index_within=index_within-1

                    #still move to the next item
                    index=index+1

                    #continue to the next round in production loop.
                    continue

                name_specific_production=list_production_names_sorted[index]

                id_specific_production=list_production_ids_sorted[index]

                roles_specific_production=[]


                index_within=0
                #while it is "on the same" production id - want to go to the next and get the role
                while list_production_ids_sorted[index_within+index]==production_id:

                    roles_specific_production.append(list_production_roles_sorted[index_within+index])

                    index_within=index_within+1

                    #
                    #if (index_within+index) > (len(list_production_ids_sorted)):
                    if (index_within+index) > (len(list_production_ids_sorted)-1): 

                        #use this to break out of printing twice also. 
                        break_status=True
                        break



                #need an output for every production
                string_roles=', '.join(roles_specific_production)

                #with link to pages
                #link:
                if break_status==False:

                    html_link='https://expansion.dh.gu.se/production/' + str(id_specific_production)

                    display(HTML('<p style="font-size:14px;font-weight:bold;margin-left:40px;"><a href=' + html_link +   ' target="_blank">' +  name_specific_production + '</a>' + ': </p>' +'<p style="font-size:14px;margin-left:40px;">' + string_roles + '</p>'))

                    #only come here once... still display html twice
                    count=count+1

                    index=index+1

            display(HTML('<p style="font-size:14px;font-weight:bold;margin-left:40px;"><a href=' + html_link +   ' target="_blank">' +  name_specific_production + '</a>' + ': </p>' +'<p style="font-size:14px;margin-left:40px;">' + string_roles + '</p>'))

            #display(HTML('<p style="font-size:14px;font-weight:bold;margin-left:40px;">Antal produktioner: ' + str(count) + ' </p>'))  
            #print(count)
            #still same output widget:
            #involvement in other organizations

            #list with productions given person:
            keys=list(Counter(list_organizations).keys()) # equals to list(set(words))
            values=list(Counter(list_organizations).values())


            #combine org and production = unique value. then pair
            list_org_prod=[]
            index=0
            for organization in list_organizations:

                org_production=organization + ': ' + list_production_names_not_org_specific[index]

                list_org_prod.append(org_production)

                index=index+1

            keys_upd=keys=list(Counter(list_org_prod).keys())

            values_upd=list(Counter(list_org_prod).values())

            #back to check how many DIFFERENT productions per organization
            keys_updated_split=[]
            for key in keys_upd:

                key_split=key.split(":")

                keys_updated_split.append(key_split[0])


            keys_upd_2=keys=list(Counter(keys_updated_split).keys())

            values_upd_2=list(Counter(keys_updated_split).values())

            display(HTML('<p style="font-size:14px;font-weight:bold;">Medverkan i alla organisationer: </p>'))


            index=0
            for value in keys_upd_2:

                #här inkludera länk
                if values_upd_2[index]==1:
                    display(HTML('<p style="font-size:14px;font-weight:bold;margin-left:40px;">' +  str(value) + ': </p>' +'<p style="font-size:14px;margin-left:40px;">' + str(values_upd_2[index]) + ' unik produktion</p>'))
                else: 

                    display(HTML('<p style="font-size:14px;font-weight:bold;margin-left:40px;">' +  str(value) + ': </p>' +'<p style="font-size:14px;margin-left:40px;">' + str(values_upd_2[index]) + ' unika produktioner</p>'))
                index=index+1

    #organization_list_of_tuples
    #now only show organizations that have any data. 
    organization = Dropdown(options = organization_list_of_tuples_only_w_productions,
                       description = 'Organization',
                       disabled= False,)

    production = Dropdown(options = [],
                       description = 'Production',
                       disabled= False,)

    person = Dropdown(options = [],
                       description = 'Person',
                       disabled= False,)

    #Create a Button widget to control execution:
    update_button = Button(description='Uppdatera',
                           disabled=False,
                           button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
                           tooltip='Click me',)

    #style of button:
    royal='#4169E1'
    update_button.style.button_color=royal
    update_button.layout.margin = '50px 100px 40px 275px' #top, right, bottom, left


    #observe - on change of organization, updated the options in production
    organization.observe(change_org, 'value')

    production.observe(change_prod, 'value')

    #update_func happening if click on the button
    update_button.on_click(update_func)

    #how to align the data to display
    h_box_1=HBox([organization, production, person])
    display(h_box_1, update_button, output_person_article, output_person_in_production, output_person_in_organization)


In [ ]:
#create table from pandas dataframe:


def render_mpl_table(data, col_width=2, row_height=0.625, font_size=14,
             header_color='#40466e', row_colors=['#f1f1f2', 'w'], edge_color='w',
             bbox=[0, 0, 1, 1], header_columns=0,
             ax=None, column_widths=[]):
    if ax is None:
        size = (np.array(data.shape[::-1]) + np.array([0, 1])) * np.array([col_width, row_height])
        fig, ax = plt.subplots(figsize=size)
        ax.axis('off')

    mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, colWidths=column_widths)

    #mpl_table = ax.table(cellText=data.values, bbox=bbox, colLabels=data.columns, colWidths=[2.5,4.5,2,2,2,2,2])

    mpl_table.auto_set_font_size(False)
    mpl_table.set_fontsize(font_size)

    for k, cell in  six.iteritems(mpl_table._cells):
        cell.set_edgecolor(edge_color)
        if k[0] == 0 or k[1] < header_columns:
            cell.set_text_props(weight='bold', color='w')
            cell.set_facecolor(header_color)
        else:
            cell.set_facecolor(row_colors[k[0]%len(row_colors) ])

    """
    if save_figs=='yes':
        fig = ax.get_figure()
        plotdir='plots'
        pngfile=station+'_figure_6'
        fig.savefig(plotdir+'/'+pngfile+'.png',dpi=100, bbox_inches='tight')
    """
    plt.show(fig)


In [ ]:
def return_statistics_variables():
    #count organizations (that also has production(s)) with articles
    
    
    max_count = len(dictionaries_organizations)+1

    f = IntProgress(min=0, max=max_count) # instantiate the bar
    display(f) # display the bar

    """
    count = 0
    while count <= max_count:
        f.value += 1 # signal to increment the progress bar
        time.sleep(.1)
        count += 1
        
    """
    count_organizations_w_articles=0

    #count organizations  with articles
    count_organizations_w_articles_all=0

    #count productions in the database
    count_productions=0

    #count the number of organization which has more than 10 productions (defined in code)
    org_with_many_productions=0

    #counting the number of organizations that also have people
    count_with_people=0

    #list of all people invlolved - then get the unique number of people involved in any of the organizations
    list_people_all_org=[]

    #this is for the overview table (generate by running next table):
    #the name of each organization that has a production
    name_organizations=[]

    #the number of people associated with each organization (added in the same order as the org names)
    list_count_unique_people_in_organization=[]

    #number of productions associated wtih each organization (added in the same order as the org names)
    list_count_productions_in_organization=[]

    #for dictionary in organizations_w_productions_dictionaries:
    #rather try to do all directly. if statement?
    for dictionary in dictionaries_organizations:

        #use the organization id to access the organization specific productions:
        org_id=dictionary['fields']['id']['value']

        organization_specific_dictionaries=get_data_by_id('productions_by_organization', org_id)

        #if one or more productions, proceed here:
        if len(organization_specific_dictionaries)>0:

            #name of organization - sometimes extra information after a comma that we want to strip
            name_org=dictionary['fields']['name']['value']
            split_name_org=name_org.split(',')
            name_organizations.append(split_name_org[0])
            
            #print(split_name_org[0])

            #how many organizations THAT ALSO HAVE PRODUCTIONS have articles associated with them?
            if dictionary['fields']['article']['value']!='':
                count_organizations_w_articles=count_organizations_w_articles+1

            #how many have people associated with the production?
            #this is the organization id
            #id_number=dictionary['fields']['id']['value']

            #total number of poductions in the database (will be all - this list of dictionary is by definition all
            #organizations that has a list)
            count_productions=count_productions+len(organization_specific_dictionaries)


            #append to list - how many productions by a certain organization:
            list_count_productions_in_organization.append(len(organization_specific_dictionaries))

            #here - how many of the productions have any people associated with them people?
            #also, and this takes a bit of time, how many "unique" people are there? Only count once.
            #want one list for each organization. 
            person_id_list=[]

            #for each organization - go into each seperate production.
            for dictionary_production in organization_specific_dictionaries:

                id_number_production=dictionary_production['fields']['id']['value']

                #the persons in each of those productions. One dictionary per production.
                production_specific_persons_dictionary_list = get_data_by_id('people_by_production', id_number_production)   

                if len(production_specific_persons_dictionary_list)>0:

                    #number of productions with people - all 
                    count_with_people=count_with_people+1

                    #if has people - want to know how many unique people
                    #use the ids, then count number unique
                    for person_dictionary in production_specific_persons_dictionary_list:

                        person_id_list.append(person_dictionary['fields']['pers_id']['value'])

                        #for the list to check the unique people in ALL organization (not just one)
                        list_people_all_org.append(person_dictionary['fields']['pers_id']['value'])

            #number of unique people in specific organization 
            count_unique_people_in_organization=len(list(Counter(person_id_list).keys()))

            list_count_unique_people_in_organization.append(count_unique_people_in_organization)

        #want to see how many organizations have many 
        if len(organization_specific_dictionaries)>10:
            org_with_many_productions=org_with_many_productions+1

        #count the number of articles in all of the organizations (not only the ones with productions)
        if dictionary['fields']['article']['value']!='':

            count_organizations_w_articles_all=count_organizations_w_articles_all+1

        
        #go to next in progressbar:
        
        f.value += 1
    #number of people associated with any organization (will only be organizations with productions
    #since the people are connected to organizations via production)
    count_unique_people_all_org=len(list(Counter(list_people_all_org).keys()))

    #number of organizations as variable
    count_organizations_all=len(dictionaries_organizations)

    #section about people

    #number of people with articles in the database
    count_people_articles=0

    for dictionary in dictionaries_people:

        #if article field not empty, count it.
        if dictionary['fields']['article']['value']!='':

            count_people_articles=count_people_articles+1

    count_people_all=len(dictionaries_people)
    
    #return the text directly - html 
    display(HTML('<h3>Överblick databasen</h3><p style="font-size:14px">Det finns <b>' +  str(len(organization_list_of_tuples_only_w_productions)) + '</b> ogranisationer som har en eller flera produktioner. <br>' +
             'Av dessa har <b>' + str(count_organizations_w_articles) + '</b> artiklar. Räknar vi alla organisationer (' + str(count_organizations_all) + ') så finns det ' + str(count_organizations_w_articles_all) + 
             ' som har artiklar. <br><b>' + str(count_productions) + '</b> produktioner har dessa organisationer sammanlagt. ' + str(org_with_many_productions) + ' har fler än 10 produktioner.<br>' + 
             'Av de ' + str(count_productions) + ' produktionerna är det <b>' + str(count_with_people) + '</b> som har några personer kopplade, totalt <b>' + str(count_unique_people_all_org) + '</b> unika personer.<br>' +
             'Totalt finns det <b>' + str(count_people_all) + '</b> personer i databasen, så de flesta används inte. <b>' + str(count_people_articles) + '</b> personer har artiklar.</p>')) 

    f.value += max_count
    return name_organizations, list_count_productions_in_organization, list_count_unique_people_in_organization


In [ ]:
def create_table_org_prod_pers():
    
    sorting_type=RadioButtons(
        options=['Alfabetisk', 'Antal produktioner', 'Antal personer'],
        value='Alfabetisk',
        description='Sortera:',
        disabled=False)
    
    #Create a Button widget to control execution:
    update_button = Button(description='Uppdatera',
        disabled=False,
        button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',)

    #style of button:
    royal='#4169E1'
    update_button.style.button_color=royal
    update_button.layout.margin = '10px 30px 40px 75px'
    
    form=VBox([sorting_type,update_button])
    
    form_out = Output()
    
    tabell_resultat = Output()
    
    def update_func(button_c):
        
        with tabell_resultat:
            
            clear_output()
        
            #possible make it so can clickable - link to the organization. Decide if look into. 
            df_org_prod_people = pd.DataFrame()
            df_org_prod_people['Organisation'] = name_organizations
            df_org_prod_people['Antal produktioner'] = list_count_productions_in_organization
            df_org_prod_people['Antal personer'] = list_count_unique_people_in_organization

            #same table - but sorted by "Antal personer" (then "Organisation")
            df_org_prod_people_people_sorted=df_org_prod_people.sort_values(['Antal personer', 'Organisation'], ascending=False)

            #same table - but sorted by "Antal produktioner" (then "Organisation")
            df_org_prod_people_production_sorted=df_org_prod_people.sort_values(['Antal produktioner', 'Organisation'], ascending=False)

            if sorting_type.value=='Alfabetisk':
                render_mpl_table(df_org_prod_people, header_columns=0, col_width=4, column_widths=[6,4,4]) 
            elif sorting_type.value=='Antal produktioner':
                render_mpl_table(df_org_prod_people_production_sorted, header_columns=0, col_width=4, column_widths=[6,4,4]) 
            else:
                render_mpl_table(df_org_prod_people_people_sorted, header_columns=0, col_width=4, column_widths=[6,4,4]) 


    update_button.on_click(update_func)
    
    
    with form_out:
    
        display(form, tabell_resultat)
        
    
    display(form_out)
    
    """
    #possible make it so can clickable - link to the organization. Decide if look into. 
    df_org_prod_people = pd.DataFrame()
    df_org_prod_people['Organisation'] = name_organizations
    df_org_prod_people['Antal produktioner'] = list_count_productions_in_organization
    df_org_prod_people['Antal personer'] = list_count_unique_people_in_organization

    #same table - but sorted by "Antal personer" (then "Organisation")
    df_org_prod_people_people_sorted=df_org_prod_people.sort_values(['Antal personer', 'Organisation'], ascending=False)

    #same table - but sorted by "Antal produktioner" (then "Organisation")
    df_org_prod_people_production_sorted=df_org_prod_people.sort_values(['Antal produktioner', 'Organisation'], ascending=False)

    render_mpl_table(df_org_prod_people_production_sorted, header_columns=0, col_width=4, column_widths=[6,4,4]) 
    """

In [ ]:
#one list of id's for every year. List of lists (40 lists)
def create_graph_new_total_org():
    style_bin = {'description_width': 'initial'}
    graph_type=RadioButtons(
        options=['Startår och slutår', 'Startår'],
        value='Startår och slutår',
        description='Organisationer med:',
        style=style_bin,
        disabled=False)
    
    
    include_production_graph=RadioButtons(
        options=['Inkludera', 'Inkludera inte'],
        value='Inkludera inte',
        description='Produktionsgraf:',
        style=style_bin,
        disabled=False)
    
    graph_type_production=RadioButtons(
        options=['Startår och slutår', 'Startår'],
        value='Startår och slutår',
        description='Produktioner med:',
        style=style_bin,
        disabled=False)
    
    #Create a Button widget to control execution:
    update_button = Button(description='Uppdatera',
        disabled=False,
        button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',)

    #style of button:
    royal='#4169E1'
    update_button.style.button_color=royal
    update_button.layout.margin = '10px 30px 40px 75px'
    
    v_box_1=VBox([graph_type, update_button])
    v_box_2=VBox([include_production_graph, graph_type_production])

    form=HBox([v_box_1, v_box_2])
    
    form_out = Output()
    
    graf_resultat = Output()
    
    def update_func(button_c):
        
        with graf_resultat:
        
            clear_output()
            
            year_list=list(range(1960,2001))

            count_new_per_year_list=[]

            count_total_per_year_list=[]
            
            list_of_lists_org_ids_per_year=[]

            for year in year_list:
                
                #for use in the next cell - otherwise takes veeery long
                list_org_ids_per_year=[]
            

                count_new_per_year=0

                count_total_per_year=0

                #check each organization
                for dictionary in dictionaries_organizations:

                    start_year_month_day=dictionary['fields']['start_date']['value']

                    start_year=int(start_year_month_day[0:4])

                    end_year_month_day=dictionary['fields']['end_date']['value']

                    end_year=int(end_year_month_day[0:4])
                    
                    org_id=dictionary['fields']['id']['value']

                    #and end_year!=0
                    #not count the end_years - end year must be greater than "year" in the next part
                    #hence it does not make sense to include here. 
                    #should change so it is not 0 - rather 2999 or something
                    if graph_type.value=='Startår och slutår':

                        if start_year==year and end_year!=0:
                            count_new_per_year=count_new_per_year+1
                            
                    else:
                        if start_year==year:

                            count_new_per_year=count_new_per_year+1

                    
                    if graph_type.value=='Startår och slutår':
                        if start_year<=year and end_year>=year and start_year!=0:
                            count_total_per_year=count_total_per_year+1
                            
                            #added for the list of lists of ids
                            list_org_ids_per_year.append(org_id)
    
                    else:
                        if start_year<=year and start_year!=0 and (end_year>=year or end_year==0):
                            count_total_per_year=count_total_per_year+1
                        
                            #added for the list of lists of ids
                            list_org_ids_per_year.append(org_id)
                        
                count_new_per_year_list.append(count_new_per_year)

                count_total_per_year_list.append(count_total_per_year)
                
                #added for the list of lists ids
                
                list_of_lists_org_ids_per_year.append(list_org_ids_per_year)
                
                


            matplotlib.rcParams.update({'font.size': 14})

            figure(num=None, figsize=(15, 8), dpi=80, facecolor='w', edgecolor='k')

            first,=plt.plot(year_list, count_new_per_year_list, 'ro')

            #get a line for number of accumulated organizations per year - assuming 0 means 
            #still exising (surely many that has to be updated - have the Excel-file for the team to go over)
            second,=plt.plot(year_list, count_total_per_year_list, 'bo')

            #plt.legend(loc='upper right')
            plt.legend([first, second], ['Nya organisationer','Antal aktiva organisationer'])

            plt.xticks(np.arange(min(year_list), max(year_list)+5, 5.0))
            plt.grid(True)
            plt.title('Organisationer per år')
            plt.xlabel('År')
            plt.ylabel('Antal organisationer')
            plt.show()
            
            if include_production_graph.value=='Inkludera':
                create_graph_total_prod(list_of_lists_org_ids_per_year)
            
            
    update_button.on_click(update_func)
    
    
    with form_out:
    
        display(form, graf_resultat)
        
        
    
    display(form_out)
    
    

In [ ]:

#this function only used along with above cell 
def create_graph_total_prod(list_of_lists_org_ids_per_year):
    
    #same as len(year_list)
    max_count=len(list_of_lists_org_ids_per_year)+1
    
    f = IntProgress(min=0, max=max_count) # instantiate the bar
    display(f) # display the bar

    
    year_list=list(range(1960,2001))

    count_total_per_year_list=[]

    index=0
    
    #maybe silly to run this loop 40 times... 40 times I check all the productions of the organization
    for year in year_list:
        
        
        list_org_id=list_of_lists_org_ids_per_year[index]

        count_new_per_year=0

        count_total_per_year=0

        #check each organization
        for org_id in list_org_id:
            
            #use organization ID to get list of productions for that organization
            productions_dictionaries=get_data_by_id('productions_by_organization', org_id)
            
            #if there are any organization - move on.
            if len(productions_dictionaries)>0:
                
                #loop over the oranization. Count all that are starting a certain year. 
                for production_dictionary in productions_dictionaries:
                    #dictionary_production['fields']['name']['value'], dictionary_production['fields']['id']['value']
                    
                    start_year_month_day=production_dictionary['fields']['start_date']['value']

                    start_year=int(start_year_month_day[0:4])
                    
                    end_year_month_day=production_dictionary['fields']['end_date']['value']

                    end_year=int(end_year_month_day[0:4])
                    
                    #if need to have start- and end year for production
                    #now need an end year - not going to be many
                    #if end_year can be 0:
                    #if start_year<=year and end_year>=year and start_year!=0:
                    if start_year<=year and start_year!=0 and (end_year>=year or end_year==0):
                    
                        
                        count_total_per_year=count_total_per_year+1
    
        count_total_per_year_list.append(count_total_per_year)
       
        #print(str(year) + ' count total productions: ' + str(count_total_per_year))
        
        index=index+1
        
        f.value+=1
       
    #create the figure:
    matplotlib.rcParams.update({'font.size': 14})

    figure(num=None, figsize=(15, 8), dpi=80, facecolor='w', edgecolor='k')

    first,=plt.plot(year_list, count_total_per_year_list, 'bo')

    #get a line for number of accumulated organizations per year - assuming 0 means 
    #still exising (surely many that has to be updated - have the Excel-file for the team to go over)
    #second,=plt.plot(year_list, count_total_per_year_list, 'bo')

    #plt.legend(loc='upper right')
    plt.legend([first], ['Antal produktioner'])

    plt.xticks(np.arange(min(year_list), max(year_list)+5, 5.0))
    plt.grid(True)
    plt.title('Produktioner per år')
    plt.xlabel('År')
    plt.ylabel('Antal produktioner')
    plt.show()
    f.value+=max_count

In [ ]:
def organization_time_line():
    from matplotlib.ticker import FormatStrFormatter
    style_bin = {'description_width': 'initial'}
    
    organization = SelectMultiple(options = organization_list_of_tuples_only_w_productions,
                   description = 'Organisationer',
                   rows=20,
                   style=style_bin,
                   disabled= False,)

    
    #Create a Button widget to control execution:
    update_button = Button(description='Uppdatera',
        disabled=False,
        button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',)

    #style of button:
    royal='#4169E1'
    update_button.style.button_color=royal
    update_button.layout.margin = '20px 50px 50px 50px'

    form=VBox([organization, update_button])
    
    form_out = Output()
    
    graf_resultat = Output()
    
    def update_func(button_c):
        
        with graf_resultat:
        
            clear_output()
            
            list_organization_ids=list(organization.value)
            


            #get list of names
            list_selected_names=[]
            for org_id in list_organization_ids:
                
                org_id_int=int(org_id)
                
                organization_dictionary= get_data_by_id_single('organization', org_id_int)
                
                name_org=organization_dictionary['fields']['name']['value']
                
                split_name_org=name_org.split(',')

                name=split_name_org[0]
                
                list_selected_names.append(name)
                
            #figure(num=None, figsize=(15, 8), dpi=80, facecolor='w', edgecolor='k')
            
            #get the organization information via id:
            
            matplotlib.rcParams.update({'font.size': 14})
            
            if len(list_organization_ids)<10:
                height=8
            elif len(list_organization_ids)>=10 and len(list_organization_ids)<=20:
                height=12
            elif len(list_organization_ids)>20 and len(list_organization_ids)<=30:
                height=16
            elif len(list_organization_ids)>30 and len(list_organization_ids)<=40:
                height=20
            elif len(list_organization_ids)>40 and len(list_organization_ids)<=50:
                height=24
            elif len(list_organization_ids)>50 and len(list_organization_ids)<=60:
                height=28
            else:
                height=30
                
            
            fig, ax = plt.subplots(figsize=(15,height))
            
            #fig=figure(num=None, figsize=(15, 8), dpi=80, facecolor='w', edgecolor='k')
            
            #updated
            x = np.linspace(1960, 2020)
            
            #change to list of names
            y = list_selected_names
            
            
            #added - top and bottom:
            ax.tick_params(axis="x", bottom=True, top=True, labelbottom=True, labeltop=True)

            for org_id in list_organization_ids:
                
                org_id_int=int(org_id)
                
                organization_dictionary= get_data_by_id_single('organization', org_id_int)
                
                start_year_month_day=organization_dictionary['fields']['start_date']['value']

                start_year=int(start_year_month_day[0:4])

                end_year_month_day=organization_dictionary['fields']['end_date']['value']

                end_year=int(end_year_month_day[0:4])
                
                #list_years=[start_year]
                
                #org name- same as before. 
                name_org=organization_dictionary['fields']['name']['value']
                
                split_name_org=name_org.split(',')

                name=split_name_org[0]

                
                html_link=html_link='https://expansion.dh.gu.se/organization/' + str(org_id_int)
                
                #same start and end year - but not when both zero
                #continue (to the next organization in the loop) so not include it in the graph
                if start_year==end_year and start_year!=0:
                    
                    display(HTML('<p style="background-color:lightsalmon;">Notera att selekterad organisation <b><a href=' + html_link + ' target="blank">' + name + '</a></b> startar och slutar samma år (' + str(start_year) + ')</p>'))
                    continue

                #if both 0 
                if end_year==0 and start_year==0:
                    
                    display(HTML('<p style="background-color:peachpuff;">Notera att selekterad organisation <b><a href=' + html_link + ' target="blank">' + name + '</a></b> varken har start- eller slutdatum</p>'))
                
                if start_year!=0:
                    if end_year==0:
                        #assuming still existing
                        end_year=2020
                        
                        display(HTML('<p style="background-color:lightyellow;">Notera att <b><a href=' + html_link + ' target="blank">' + name + '</a></b> har slutår 0, vilket är satt till 2020 i grafen</p>'))
                        
                    #one horizontal line for each of the organizations that has a start_year
                    ax.hlines(y=name, xmin=start_year, xmax=end_year, linewidth=3, color='b')
                 
                    ax.xaxis.set_major_formatter(FormatStrFormatter('%.0f'))
                    #ax.set_xticklabels(tick_labels.astype(int))
    
            plt.title('Tidslinje för databasens organisationer' + '\n')
            plt.grid(True)
            plt.show()
            
    update_button.on_click(update_func)
    
    
    with form_out:
    
        display(form, graf_resultat)     
    
    display(form_out)

In [ ]:

def personer_via_org_roll():
    #function describing what will happen when the user changes the organization:
    #populate the role dropdown with all role(s) that the organization have
    

    #function describing what will happen once change the production (once the productions dropdown 
    #has been populated depening on what organization the user choose):
    #get all the people that was part of the selected production into person dropdown.

    #output widgets - to be able to clear only that specific widget when click update_buton

    output_productions = Output()
    #output_person_in_production= Output()

    #output_person_in_organization= Output()

    #output_person_production_specific= Output()
    
    list_roles=fetch_data_roles()

    #what happend when click the button:
    #here could organize different output widgets
    def update_func(button_c):


        #start with an article about the selected person (will disply if one is selected)
        with output_productions:
            
            clear_output()

            organization_ids=organization.value
        
                        
            lat_list=[]
            lon_list=[]
            
            name_list=[]
            list_name_places_all=[]
            #when a production has a role - add it to this list.
            #then get all the events by these productions. Put on map. 
            for organization_id in organization_ids:
                
                data_org=get_data_by_id_single('organization', organization_id)
                
                org_name=data_org['fields']['name']['value']
                
                html_link='https://expansion.dh.gu.se/organization/' + str(organization_id)

                display(HTML('<p style="font-size:16px;font-weight:bold;"><a href=' + html_link + ' target="blank">' + org_name+ '</a>: </p>'))

                list_dictionaries_productions=get_data_by_id('productions_by_organization', id_number=organization_id)

                list_productions_id_by_organization=[]
                list_productions_names_by_organization=[]
                for dictionary_production in list_dictionaries_productions:

                    list_productions_id_by_organization.append(dictionary_production['fields']['id']['value'])
                    list_productions_names_by_organization.append(dictionary_production['fields']['name']['value'])
                    
                index=0

                for production_id in list_productions_id_by_organization:
                    
                    #only display the production name the first time
                    first=True

                    production_id=str(production_id)
                    
                    production_name=list_productions_names_by_organization[index]
                    

                    list_dictionaries_people=get_data_by_id('people_by_production', id_number=production_id)

                    for dictionary_person_by_production in list_dictionaries_people:

                        role=dictionary_person_by_production['fields']['pers_role_id']['extra']


                        if role in roles.value:
                            
                            if first==True:
                                
                                #get list of places here!
                                #only display the production name and link once
                                first=False
                                
                                html_link='https://expansion.dh.gu.se/production/' + str(production_id)
                
                                events_by_production=get_data_by_id('events_by_production', production_id)
                    
                                if len(events_by_production)>0:
                                    list_name_places=[]
                                    for event in events_by_production:

                                        place_id=event['fields']['place_id']['value']

                                        place_information=get_data_by_id_single('place', place_id)

                                        lon_lat= place_information['fields']['geom']['value']

                                        lon_lat_split=lon_lat.split(' ')
                            
                                        lat_list.append(float(lon_lat_split[1]))
                                        lon_list.append(float(lon_lat_split[0]))
                                

                                        name_place=place_information['fields']['name']['value']
                                        
        
                                        list_name_places.append(name_place)
                                        list_name_places_all.append(name_place)

                                    unique_places_production_list=list(Counter(list_name_places).keys())

                                    unique_places_production_list_string = ', '.join(unique_places_production_list)
                            
                                    display(HTML('<p style="font-size:14px;margin-left:40px;"><b><a href=' + html_link + ' target="blank">' + production_name+ '</a></b> (' + unique_places_production_list_string + '):</p>'))
    
                                else:
                    
                                    display(HTML('<p style="font-size:14px;margin-left:40px;font-weight:bold;"><a href=' + html_link + ' target="blank">' + production_name+ ':</p>'))
    
                                
                            
                            name_sex=dictionary_person_by_production['fields']['pers_id']['extra']
                            
                            person_id=dictionary_person_by_production['fields']['pers_id']['value']
                            split_name_sex=name_sex.split(',')

                            #remove what sex
                            if len(split_name_sex)>2:
                                del split_name_sex[-1]
                            
                            if len(split_name_sex)>1:
                                name_first_last=split_name_sex[1] + ' ' + split_name_sex[0]
                            else:
                                name_first_last=split_name_sex
                            
                            html_link='https://expansion.dh.gu.se/person/' + str(person_id)
                                
                            display(HTML('<p style="font-size:14px;margin-left:80px;"><a href=' + html_link + ' target="blank">' + name_first_last + '</a></p>'))

                            
                    #move the the next production name
                    index=index+1
                 
            if len(lat_list)>0:
                m = folium.Map(location=[57.650002, 12.016667])
                tooltip = 'Klicka!'
                
                index=0
                for lat in lat_list:
                    #print(lat_lon)
                    #print(type(lat_lon[0]))
                    lon=lon_list[index]
                    folium.Marker([lat, lon], popup='<i>' + list_name_places_all[index] + '</i>', tooltip=tooltip).add_to(m)
                    index=index+1
                display(m)

         
    #so show the whole text
    style_bin = {'description_width': 'initial'}
    
    
    
    roles = SelectMultiple(options = list_roles,
                   description = 'Roller',
                   rows=20,
                   style=style_bin,
                   disabled= False,)
    
    organization = SelectMultiple(options = organization_list_of_tuples_only_w_productions,
                   description = 'Organisationer',
                   rows=20,
                   style=style_bin,
                   disabled= False,)



    #Create a Button widget to control execution:
    update_button = Button(description='Uppdatera',
                           disabled=False,
                           button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
                           tooltip='Click me',)

    #style of button:
    royal='#4169E1'
    update_button.style.button_color=royal
    update_button.layout.margin = '50px 100px 40px 275px' #top, right, bottom, left


    #observe - on change of organization, updated the options in production
    #organization.observe(change_org, 'value')

    #production.observe(change_prod, 'value')

    #update_func happening if click on the button
    update_button.on_click(update_func)

    #how to align the data to display
    h_box_1=HBox([roles, organization])
    display(h_box_1, update_button, output_productions)
